## Importar Bibliotecas Necessárias

Neste bloco, vamos importar todas as bibliotecas necessárias para realizar operações no banco de dados SQLite e manipular os dados.


In [20]:
import sqlite3
import pandas as pd
import os
import csv
from tabulate import tabulate
from IPython.display import display, HTML
import time
import math

## Salvando DataFrame no Banco de Dados SQLite

O bloco de código abaixo é responsável por salvar o DataFrame `df` em um banco de dados relacional SQLite. Utilizando a biblioteca `sqlite3`, o DataFrame será salvo em uma tabela no banco de dados. Caso haja algum erro durante o processo, uma mensagem será exibida. Além disso, o código também informa a quantidade de linhas que foram adicionadas no banco de dados e as colunas inseridas.

In [23]:
# Caminho para o CSV
csv_path = os.path.join('data', 'temas_ambientais.csv')

# Caminho para o banco de dados SQLite
db_path = os.path.join('banco', 'projeto_big_data.db')

# Nome da tabela no SQLite
tabela_sqlite = "temas_ambientais"

# Função para inferir o tipo de dado
def inferir_tipo(valor):
    try:
        int(valor)
        return "INTEGER"
    except ValueError:
        try:
            float(valor)
            if '.' in valor:
                return "DOUBLE"
            return "REAL"
        except ValueError:
            return "TEXT"

try:
    # Conectar ao banco de dados SQLite
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Ler o CSV e criar a tabela no SQLite
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=';')
        cabecalho = next(csv_reader)  # Ler o cabeçalho

        # Inferir os tipos de dados das primeiras linhas
        primeira_linha = next(csv_reader)
        tipos = [inferir_tipo(valor) for valor in primeira_linha]
        
        # Criar string de criação de tabela com tipos inferidos
        colunas = ", ".join([f"{col} {tipo}" for col, tipo in zip(cabecalho, tipos)])
        create_table_sql = f"CREATE TABLE IF NOT EXISTS {tabela_sqlite} ({colunas})"
        cursor.execute(create_table_sql)

        # Reiniciar o csv_reader para incluir a primeira linha nos dados
        csvfile.seek(0)
        csv_reader = csv.reader(csvfile, delimiter=';')
        next(csv_reader)  # Pular o cabeçalho novamente

        # Inserir as linhas do CSV na tabela SQLite em transações
        bloco_tamanho = 1000  # Ajuste conforme necessário
        bloco = []
        for linha in csv_reader:
            bloco.append(linha)
            if len(bloco) >= bloco_tamanho:
                cursor.executemany(f"INSERT INTO {tabela_sqlite} VALUES ({', '.join(['?' for _ in cabecalho])})", bloco)
                conn.commit()
                bloco = []
        
        # Inserir quaisquer linhas restantes
        if bloco:
            cursor.executemany(f"INSERT INTO {tabela_sqlite} VALUES ({', '.join(['?' for _ in cabecalho])})", bloco)
            conn.commit()

    print("DataFrame salvo no SQLite com sucesso.")

    # Verificar dados salvos
    cursor.execute(f"SELECT COUNT(*) FROM {tabela_sqlite}")
    numero_de_linhas = cursor.fetchone()[0]
    print(f"Número de linhas adicionadas: {numero_de_linhas}")

except Exception as e:
    print(f"Erro ao salvar no SQLite: {e}")

finally:
    # Fechar a conexão com o SQLite
    if conn:
        conn.close()
    print("Conexão com o SQLite fechada com sucesso.")


DataFrame salvo no SQLite com sucesso.
Número de linhas adicionadas: 6839104
Conexão com o SQLite fechada com sucesso.


## Conectar ao Banco de Dados SQLite

Este bloco de código estabelece uma conexão com o banco de dados SQLite onde nosso CSV foi salvo.


In [2]:
# Caminho para o banco de dados SQLite
db_path = os.path.join('banco', 'projeto_big_data.db')

# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
print("Conexão estabelecida com sucesso.")

Conexão estabelecida com sucesso.


## Verificar as Tabelas no Banco de Dados

Neste bloco, vamos listar todas as tabelas existentes no banco de dados SQLite.


In [3]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()
print("Tabelas no banco de dados:", tabelas)


Tabelas no banco de dados: [('temas_ambientais',)]


## Consulta para Mostrar os Tipos de Cada Coluna

Neste bloco de código, vamos utilizar uma consulta SQL para listar os tipos de cada coluna na tabela `temas_ambientais`.



In [4]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para listar os tipos de cada coluna na tabela
query = "PRAGMA table_info(temas_ambientais);"

# Executar a consulta
cursor.execute(query)
resultados = cursor.fetchall()

# Exibir os resultados de forma tabulada
colunas_tabela_info = ["ID", "Nome", "Tipo", "Não Nulo", "Valor Padrão", "Chave Primária"]
print(tabulate(resultados, headers=colunas_tabela_info, tablefmt='psql'))

# Fechar a conexão com o banco de dados SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")


+------+------------------------------------------+---------+------------+----------------+------------------+
|   ID | Nome                                     | Tipo    |   Não Nulo | Valor Padrão   |   Chave Primária |
|------+------------------------------------------+---------+------------+----------------+------------------|
|    0 | uf                                       | TEXT    |          0 |                |                0 |
|    1 | municipio                                | TEXT    |          0 |                |                0 |
|    2 | codigo_ibge                              | INTEGER |          0 |                |                0 |
|    3 | area_do_imovel                           | DOUBLE  |          0 |                |                0 |
|    4 | registro_car                             | TEXT    |          0 |                |                0 |
|    5 | situacao_cadastro                        | TEXT    |          0 |                |                0 |
|

## Visualizar os Dados da Tabela

Aqui, vamos visualizar os primeiros registros da tabela `temas_ambientais`.


In [6]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query para selecionar os primeiros 10 registros
query = "SELECT * FROM temas_ambientais LIMIT 5;"

# Executar a query
cursor.execute(query)
resultados = cursor.fetchall()

# Obter os nomes das colunas
colunas = [description[0] for description in cursor.description]

# Formatar os resultados como uma tabela HTML
tabela_html = "<table>"
tabela_html += "<tr>" + "".join([f"<th>{col}</th>" for col in colunas]) + "</tr>"
for linha in resultados:
    tabela_html += "<tr>" + "".join([f"<td>{item}</td>" for item in linha]) + "</tr>"
tabela_html += "</table>"

# Exibir a tabela HTML
display(HTML(tabela_html))

# Fechar a conexão com o SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")


uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,area_preservacao_permanente,area_nao_classificada,solicitacao_adesao_pra,latitude,longitude,data_inscricao,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
GO,Nazário,5214408,119.6326,GO-5214408-3AEF2043582E40238C0F84A553686CA7,AT,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",119.6326,6.43202795367569,6.432,5.29751084282892,0.00102295610380904,Sim,-16.5923058689987,-49.9019017039191,2014-05-07 16:01:44.305,,112.301149046683,0,IRU,5.4378,0,0.0,0.0,0,2014-05-07 16:01:44.305
SC,Meleiro,4210803,7.534,SC-4210803-BC127B0EC8DB49AC9D46D723286241A2,AT,"Aguardando análise, não passível de revisão de dados",7.534,5.51785410336982,0.0,0.0,0.00159349317984888,Nao,-28.7930798512303,-49.6472023744097,2014-05-07 16:02:02.915,,2.01445162492469,0,IRU,0.4186,0,1.506,0.0,0,2014-05-07 16:02:02.915
GO,Nova Roma,5214903,19.4883,GO-5214903-7F58049BD79046E9A904CC81C5AC177A,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",19.36,19.4882633569717,3.872,0.0,0.0,Nao,-13.6370551503248,-47.0339670619739,2014-05-07 16:06:15.777,,0.0,0,IRU,0.2784,0,0.0,0.0,0,2014-05-07 16:06:15.777
GO,Santa Helena de Goiás,5219308,22.934,GO-5219308-6478196E75CF4F65800ACA0758575820,PE,"Analisado com pendências, aguardando retificação",22.88,1.5021496193707,1.5028,0.568927694143718,18.2335997514784,Sim,-17.8327877579625,-50.6015145422339,2014-05-07 17:49:36.938,,3.08289042473435,0,IRU,1.1467,0,0.0,0.0,0,2014-05-07 17:49:36.938
PR,Cornélio Procópio,4106407,10.956,PR-4106407-0F06081500254BE3A479EE8EFFDD5319,AT,Em análise,10.956,0.0,0.0,0.0,0.161657419734154,Nao,-23.1841101335722,-50.6715645967419,2014-05-07 17:52:55.333,,10.7942669028953,0,IRU,0.6087,0,0.0,0.0,0,2014-05-07 17:52:55.333


Conexão com o SQLite fechada com sucesso.


## Primeira Consulta 

Recupere a soma de área que é a coluna "Area Do Imovel" para todas as propriedades agrícolas que pertecem ao MS e MT que estão na coluna "Uf". Ordene o resultado em ordem descrescente.


In [12]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para recuperar a soma de área para MS e MT
query = """
SELECT "uf", SUM("area_do_imovel") AS soma_area
FROM temas_ambientais
WHERE "Uf" IN ('MS', 'MT')
GROUP BY "Uf"
ORDER BY soma_area DESC;
"""

# Medir o tempo de execução da consulta
tempo_inicio = time.time()

# Executar a consulta
cursor.execute(query)
resultados = cursor.fetchall()

tempo_execucao = time.time() - tempo_inicio

# Obter os nomes das colunas
colunas = [description[0] for description in cursor.description]

# Formatar os resultados como uma tabela HTML
tabela_html = "<table>"
tabela_html += "<tr>" + "".join([f"<th>{col}</th>" for col in colunas]) + "</tr>"
for linha in resultados:
    tabela_html += "<tr>" + "".join([f"<td>{item}</td>" for item in linha]) + "</tr>"
tabela_html += "</table>"

# Exibir a tabela HTML
display(HTML(tabela_html))

# Exibir o tempo de execução da consulta
print(f"Tempo de execução da consulta: {tempo_execucao:.2f} segundos")

# Fechar a conexão com o banco de dados SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")

uf,soma_area
MT,84587089.26979955
MS,36321882.66120003


Tempo de execução da consulta: 6.16 segundos
Conexão com o SQLite fechada com sucesso.


## Segunda Consulta 

Filtrar todas as propriedades que pertencem à região sudeste do Brasil. Os estados da região sudeste são SP, RJ, MG e ES, conforme indicado na coluna "Uf".


In [24]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para filtrar as propriedades da região sudeste
query = """
SELECT *
FROM temas_ambientais
WHERE "uf" IN ('SP', 'RJ', 'MG', 'ES') LIMIT 5;
"""

# Medir o tempo de execução da consulta
tempo_inicio = time.time()

# Executar a consulta
cursor.execute(query)
resultados = cursor.fetchall()

tempo_execucao = time.time() - tempo_inicio

# Obter os nomes das colunas
colunas = [description[0] for description in cursor.description]

# Formatar os resultados como uma tabela HTML
tabela_html = "<table>"
tabela_html += "<tr>" + "".join([f"<th>{col}</th>" for col in colunas]) + "</tr>"
for linha in resultados:
    tabela_html += "<tr>" + "".join([f"<td>{item}</td>" for item in linha]) + "</tr>"
tabela_html += "</table>"

# Exibir a tabela HTML
display(HTML(tabela_html))

# Exibir o tempo de execução da consulta
print(f"Tempo de execução da consulta: {tempo_execucao:.2f} segundos")

# Fechar a conexão com o banco de dados SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")


uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,area_preservacao_permanente,area_nao_classificada,solicitacao_adesao_pra,latitude,longitude,data_inscricao,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
RJ,Itaperuna,3302205,6.0851,RJ-3302205-169354AD04A446EC9AB27A79E4310489,PE,"Analisado com pendências, aguardando retificação",5.931,0.0,1.4792,1.61754906789992,5.83886665807348,Sim,-21.3214432276222,-41.80444759989,2014-05-08 11:02:20.455,,0.0501235222115647,0.1619,IRU,0.2028,0,0.0,0.0,4.33622302031857,2014-05-08 11:02:20.455
RJ,Itaperuna,3302205,44.4215,RJ-3302205-1DEA0210F20F4ECEBADF8585534E9177,PE,"Analisado com pendências, aguardando retificação",43.3195,0.0,8.8523,11.0546352649015,40.9607595756084,Sim,-21.3400673460579,-41.6868689432052,2014-05-08 12:03:58.024,,0.113328063728078,1.1514,IRU,1.4807,0,0.0,0.0,31.9864053125698,2014-05-08 12:03:58.024
RJ,Itaperuna,3302205,0.4112,RJ-3302205-8ACBE3784F13410A9835E85EB743E95B,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",0.4118,0.0,0.084,0.330845682804729,0.0880945837435553,Sim,-21.152467193082,-41.9706779504834,2014-05-08 17:23:27.499,,0.262559143783042,0,IRU,0.0137,0,0.0,0.0,0,2014-05-08 17:23:27.499
RJ,Rio Bonito,3304300,93.9777,RJ-3304300-66B9CE428807488391B38ACFBDCF1DBD,AT,Em análise,93.9777,38.3784881475703,20.2016,12.7393563370317,52.554476232823,Sim,-22.6971257840924,-42.5851337250361,2014-05-08 17:37:03.403,,0.0,0,IRU,6.7127,0,0.0,0.0,0,2014-05-08 17:37:03.403
MG,Sabará,3156700,2.39,MG-3156700-A61D7CC1D7994BFE939ACC2D69CCA87A,AT,"Aguardando análise, não passível de revisão de dados",2.39,0.0,0.0,0.0,2.38938456031024,Nao,-19.9113811223832,-43.8032291425663,2014-05-10 09:04:55.377,,0.0,0,IRU,0.34,0,0.0,0.0,0,2014-05-10 09:04:55.377


Tempo de execução da consulta: 0.97 segundos
Conexão com o SQLite fechada com sucesso.


## Terceira Consulta 

Calcular quantas propriedades foram cadastradas por ano, utilizando a coluna `data_inscricao` e agrupando por ano. Exibiremos somente as colunas que não são vazias.


In [17]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para calcular quantas propriedades foram cadastradas por ano
query = """
SELECT strftime('%Y', data_inscricao) AS ano, COUNT(*) AS total_propriedades
FROM temas_ambientais
WHERE data_inscricao IS NOT NULL
GROUP BY ano
HAVING ano IS NOT NULL
ORDER BY ano;
"""

# Medir o tempo de execução da consulta
tempo_inicio = time.time()

# Executar a consulta
cursor.execute(query)
resultados = cursor.fetchall()

tempo_execucao = time.time() - tempo_inicio

# Obter os nomes das colunas
colunas = [description[0] for description in cursor.description]

# Formatar os resultados como uma tabela HTML
tabela_html = "<table>"
tabela_html += "<tr>" + "".join([f"<th>{col}</th>" for col in colunas]) + "</tr>"
for linha in resultados:
    tabela_html += "<tr>" + "".join([f"<td>{item}</td>" for item in linha]) + "</tr>"
tabela_html += "</table>"

# Exibir a tabela HTML
display(HTML(tabela_html))

# Exibir o tempo de execução da consulta
print(f"Tempo de execução da consulta: {tempo_execucao:.2f} segundos")

# Fechar a conexão com o banco de dados SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")


ano,total_propriedades
2013,743
2014,227468
2015,1240178
2016,1828786
2017,794779
2018,724856
2019,746570
2020,463322
2021,443942
2022,368448


Tempo de execução da consulta: 14.13 segundos
Conexão com o SQLite fechada com sucesso.


## Quarta Consulta

Calcular o percentual médio de área remanescente de vegetação nativa (`area_remanescente_vegetacao_nativa`) em relação à área total do proprietário (`area_do_imovel`).


In [18]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para calcular o percentual médio de área remanescente de vegetação nativa
query = """
SELECT AVG((area_remanescente_vegetacao_nativa / area_do_imovel) * 100) AS percentual_medio
FROM temas_ambientais
WHERE area_remanescente_vegetacao_nativa IS NOT NULL AND area_do_imovel IS NOT NULL;
"""

# Medir o tempo de execução da consulta
tempo_inicio = time.time()

# Executar a consulta
cursor.execute(query)
resultados = cursor.fetchall()

tempo_execucao = time.time() - tempo_inicio

# Obter os nomes das colunas
colunas = [description[0] for description in cursor.description]

# Formatar os resultados como uma tabela HTML
tabela_html = "<table>"
tabela_html += "<tr>" + "".join([f"<th>{col}</th>" for col in colunas]) + "</tr>"
for linha in resultados:
    tabela_html += "<tr>" + "".join([f"<td>{item:.2f}%</td>" for item in linha]) + "</tr>"
tabela_html += "</table>"

# Exibir a tabela HTML
display(HTML(tabela_html))

# Exibir o tempo de execução da consulta
print(f"Tempo de execução da consulta: {tempo_execucao:.2f} segundos")

# Fechar a conexão com o banco de dados SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")


percentual_medio
17.93%


Tempo de execução da consulta: 5.99 segundos
Conexão com o SQLite fechada com sucesso.


## Quinta Consulta 

Contar o número de propriedades rurais por estado, utilizando a coluna `uf`.


In [19]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para contar o número de propriedades rurais por estado
query = """
SELECT "uf", COUNT(*) AS total_propriedades
FROM temas_ambientais
GROUP BY "uf"
ORDER BY total_propriedades DESC;
"""

# Medir o tempo de execução da consulta
tempo_inicio = time.time()

# Executar a consulta
cursor.execute(query)
resultados = cursor.fetchall()

tempo_execucao = time.time() - tempo_inicio

# Obter os nomes das colunas
colunas = [description[0] for description in cursor.description]

# Formatar os resultados como uma tabela HTML
tabela_html = "<table>"
tabela_html += "<tr>" + "".join([f"<th>{col}</th>" for col in colunas]) + "</tr>"
for linha in resultados:
    tabela_html += "<tr>" + "".join([f"<td>{item}</td>" for item in linha]) + "</tr>"
tabela_html += "</table>"

# Exibir a tabela HTML
display(HTML(tabela_html))

# Exibir o tempo de execução da consulta
print(f"Tempo de execução da consulta: {tempo_execucao:.2f} segundos")

# Fechar a conexão com o banco de dados SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")


uf,total_propriedades
BA,1012499
MG,989446
RS,606454
PR,496164
SP,408528
SC,374918
PE,339371
CE,314866
PA,279598
MA,273642


Tempo de execução da consulta: 8.58 segundos
Conexão com o SQLite fechada com sucesso.


## Sexta Consulta 

Calcular a distância entre a maior propriedade (coluna `area_do_imovel`) e Brasília, utilizando as coordenadas de centróide da propriedade nas colunas `latitude` e `longitude`.

**Usaremos a fórmula de Haversine para calcular a distância entre dois pontos geográficos.**


In [21]:
# Função para calcular a distância usando a fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    # Converter graus para radianos
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Fórmula de Haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Raio da Terra em quilômetros (varia entre 6,371 e 6,378 km)
    r = 6371.0
    return c * r

# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para obter a propriedade com a maior área
query_maior_propriedade = """
SELECT latitude, longitude, area_do_imovel
FROM temas_ambientais
ORDER BY area_do_imovel DESC
LIMIT 1;
"""

# Medir o tempo de execução da consulta
tempo_inicio = time.time()

# Executar a consulta
cursor.execute(query_maior_propriedade)
maior_propriedade = cursor.fetchone()

# Coordenadas de Brasília (lat: -15.8267, lon: -47.9218)
latitude_brasilia = -15.8267
longitude_brasilia = -47.9218

# Calcular a distância usando a fórmula de Haversine
distancia = haversine(maior_propriedade[0], maior_propriedade[1], latitude_brasilia, longitude_brasilia)

tempo_execucao = time.time() - tempo_inicio

# Formatar o resultado como uma tabela HTML
tabela_html = f"""
<table>
    <tr><th>Latitude</th><td>{maior_propriedade[0]}</td></tr>
    <tr><th>Longitude</th><td>{maior_propriedade[1]}</td></tr>
    <tr><th>Área do Imóvel</th><td>{maior_propriedade[2]}</td></tr>
    <tr><th>Distância até Brasília (km)</th><td>{distancia:.2f} km</td></tr>
</table>
"""

# Exibir a tabela HTML
display(HTML(tabela_html))

# Exibir o tempo de execução da consulta
print(f"Tempo de execução da consulta: {tempo_execucao:.2f} segundos")

# Fechar a conexão com o banco de dados SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")


Latitude,-5.47561883060412
Longitude,-68.8881881087452
Área do Imóvel,2420078.7379
Distância até Brasília (km),2560.56 km


Tempo de execução da consulta: 5.62 segundos
Conexão com o SQLite fechada com sucesso.


## Sétima Consulta 

Calcular a média de todas as propriedades (`area_do_imovel`) e, em seguida, determinar quantas propriedades por estado estão acima da média (`uf`).

**Vamos usar GROUP BY para agrupar os estados e contar as propriedades acima da média.**


In [22]:
# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para calcular a média de todas as propriedades
query_media = """
SELECT AVG(area_do_imovel) AS media_area
FROM temas_ambientais;
"""

# Medir o tempo de execução da consulta
tempo_inicio = time.time()

# Executar a consulta para obter a média
cursor.execute(query_media)
media_area = cursor.fetchone()[0]

# Consulta SQL para contar as propriedades por estado que estão acima da média
query_acima_media = f"""
SELECT uf, COUNT(*) AS total_acima_media
FROM temas_ambientais
WHERE area_do_imovel > {media_area}
GROUP BY uf
ORDER BY total_acima_media DESC;
"""

# Executar a consulta para obter as propriedades acima da média por estado
cursor.execute(query_acima_media)
resultados = cursor.fetchall()

tempo_execucao = time.time() - tempo_inicio

# Obter os nomes das colunas
colunas = [description[0] for description in cursor.description]

# Formatar os resultados como uma tabela HTML
tabela_html = "<table>"
tabela_html += "<tr>" + "".join([f"<th>{col}</th>" for col in colunas]) + "</tr>"
for linha in resultados:
    tabela_html += "<tr>" + "".join([f"<td>{item}</td>" for item in linha]) + "</tr>"
tabela_html += "</table>"

# Exibir a tabela HTML
display(HTML(tabela_html))

# Exibir o tempo de execução da consulta
print(f"Tempo de execução da consulta: {tempo_execucao:.2f} segundos")

# Fechar a conexão com o banco de dados SQLite
conn.close()
print("Conexão com o SQLite fechada com sucesso.")


uf,total_acima_media
MG,107875
PA,86698
MT,78569
GO,61385
BA,50508
SP,45229
MA,42202
RS,40274
MS,33762
TO,33389


Tempo de execução da consulta: 9.86 segundos
Conexão com o SQLite fechada com sucesso.
